<a href="https://colab.research.google.com/github/Nofangagah/DS-final-project/blob/main/Final_project_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Science Practicum Final project**

In [5]:
import requests
import os
import json

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Buat folder untuk menyimpan dataset kanker kulit di Google Drive
os.makedirs('/content/drive/MyDrive/kanker_kulit_datasets', exist_ok=True)

# Endpoint API
url = "https://api.isic-archive.com/api/v2/images"

# Parameter API
params = {
    "limit": 1,             # Batasi jumlah gambar (sesuaikan dengan kebutuhan)
    "offset": 0,            # Posisi awal
    "sort": "created",      # Urutkan berdasarkan waktu
    "sortdir": -1           # Urutkan dari yang terbaru
}

# Kirim permintaan ke API
response = requests.get(url, params=params)

# Debug respons API
print(f"Status code: {response.status_code}")
if response.status_code != 200:
    print("Gagal mengambil data dari API.")
    exit()

# Parsing JSON
try:
    data = response.json()  # Parsing JSON
    images = data.get('results', [])  # Ambil key 'results' sebagai daftar gambar
    if not images:
        print("Data kosong. Tidak ada gambar yang ditemukan.")
        exit()
    print(f"Berhasil mendapatkan {len(images)} gambar.")
except json.JSONDecodeError:
    print("Gagal parsing JSON. Periksa respons API.")
    exit()

# Loop untuk mendownload gambar dan metadata
for idx, image in enumerate(images):
    try:
        # Ambil informasi gambar dan metadata
        image_id = image['isic_id']  # ID gambar unik
        image_url = image['files']['full']['url']  # URL gambar
        metadata = {
            'isic_id': image_id,
            'description': image.get('description', 'No description available'),
            'license': image.get('copyright_license', 'No license available'),
            'attribution': image.get('attribution', 'No attribution available'),
            'acquisition': image.get('metadata', {}).get('acquisition', {}),
            'clinical': image.get('metadata', {}).get('clinical', {})
        }

        # Buat folder berdasarkan ID gambar di Google Drive
        image_folder = os.path.join('/content/drive/MyDrive/kanker_kulit_datasets', image_id)
        os.makedirs(image_folder, exist_ok=True)

        print(f"Mengunduh {image_id} dari {image_url}...")

        # Unduh gambar dan simpan ke folder ID gambar di Google Drive
        img_data = requests.get(image_url).content
        with open(os.path.join(image_folder, f'{image_id}.jpg'), 'wb') as f:
            f.write(img_data)

        # Simpan metadata ke file JSON di folder ID gambar di Google Drive
        with open(os.path.join(image_folder, f'{image_id}_metadata.json'), 'w') as metadata_file:
            json.dump(metadata, metadata_file, indent=4)

    except KeyError as e:
        print(f"Field tidak ditemukan pada item {idx}: {e}")
    except Exception as e:
        print(f"Error pada item {idx}: {e}")


Mounted at /content/drive
Status code: 200
Berhasil mendapatkan 1 gambar.
Mengunduh ISIC_9536278 dari https://content.isic-archive.com/a70630df-48e5-4a69-b8fd-f1f9f5143caa/5d8fee0c-54ba-4686-808b-0ecc22d23b96.jpg?Expires=1733443200&Signature=PenGRqwT84ea2x4DBRp5lSTlCklReh2htjPoiRY1mvJT9tW3QXhDeqpjbLyLKUgbjBrApeIXQGsgJfGng2q5mQwot2kVlJOY7lKbC9~zB6PfhvvbP41cEadZAJQE8FLftNr00FbeDMVg81Z3w5Z2euy14GXd72S1Pwwtk52wLcku5ppE0jgOZkFuR2lBrId6DygK5BHuMhOrhch2bj-DnqiH9acToiwm3LVH4qVRtqMdLKcizGBziCJa~9-O5tql~RHjiWVkp2J5gb6v8lMs-zWEVUTF6X~BwGPXRzkD2Ft0HjZjE-wVUrYKnxwbAHegC6uiNw5GB-aPNJomaYG0Mg__&Key-Pair-Id=K3KFHCM130RXTL...
